In [1]:
!pip3 install langchain
!pip3 install ollama
!pip3 install datasets peft langchain
!pip3 install sentence-transformers==2.2.2
!pip3 install InstructorEmbedding
!pip3 install faiss-cpu
!pip install chainlit

In [1]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

In [3]:
import ollama
response = ollama.chat(model='mistrallite', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

 The answer to this question has eluded humankind for thousands of years, and it wasn't until the late 19th century that scientists figured out the real reason. The common explanation you might have heard growing up—that the molecules in the atmosphere scatter short-wavelength light more than long wavelengths, making the sky appear blue—is actually only partially true.

To understand why, let's first examine what's happening at a fundamental level. Sunlight is made up of all colors in the electromagnetic spectrum. Blue light has a shorter wavelength than other colors in the visible spectrum, like red and green, but it's not the shortest. That title belongs to ultraviolet light, which humans can't see.

When sunlight enters Earth's atmosphere, molecules (most notably oxygen and nitrogen) start absorbing different parts of the spectrum depending on their energy levels. Ultraviolet light is absorbed by ozone in the stratosphere, leaving behind shorter wavelengths like blue, green, and vio

In [2]:
prompt_template = """
### [INST] Instruction: Answer the question based only on the knowledge provided in the given context. Here is the context to help:
You are PolicyBot, an AI assistant for the car insurance company.
Your role is to assist customers by providing information on their insurance policies, helping them understand their coverage, and guiding them through the claim process.
Ask relevant questions to understand the customer's needs better and then provide accurate answers according to the context given: {context}.
Please be precise with numerical values like policy numbers, coverage limits, and contact details.
Use bullet points if the user asks to "list", "show", or "display" policy details or coverage options.
Refer to previous conversations with the customer to offer personalized assistance.
### QUESTION:
{question} [/INST]

  """

In [3]:
from langchain_community.llms import Ollama
ollama = Ollama(model="mistrallite")

In [4]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
# from langchain.document_transformers import Html2TextTransformer
# from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

In [6]:
tweet_prompt = PromptTemplate.from_template(prompt_template)
tweet_chain = LLMChain(llm=ollama, prompt = tweet_prompt, verbose=True)

In [19]:
!pip3 install unstructured[pdf]

In [7]:
import pandas as pd
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
# # Load the dataset into a pandas DataFrame for easy manipulation
# data = pd.read_csv("/content/myntra_products_catalog.csv")

# # Display the first few entries to understand the data structure
# print(data.head())

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader
loader =  OnlinePDFLoader(r"C:\Users\asifhussain\Downloads\policy-booklet-0923.pdf")
data = loader.load()
instructor_embeddings = HuggingFaceInstructEmbeddings()

load INSTRUCTOR_Transformer


C:\Users\asifhussain\anaconda3\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


max_seq_length  512


In [25]:
!pip3 install faiss-cpu

In [8]:
vectordb = FAISS.from_documents(documents=data, embedding=instructor_embeddings)
retriever = vectordb.as_retriever(score_threshold = 0.7)


In [30]:
# vectordb = FAISS.load_local('my_vector',instructor_embeddings,allow_dangerous_deserialization=True)
# retriever = vectordb.as_retriever(score_threshold = 0.7)

RuntimeError: Error in __cdecl faiss::FileIOReader::FileIOReader(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:68: Error: 'f' failed: could not open my_vector\index.faiss\index.faiss for reading: No such file or directory

In [9]:
rag_chain = ( {"context": retriever , "question": RunnablePassthrough()} | tweet_chain )

In [10]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [11]:
from langchain.memory import  ChatMessageHistory,ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory
message_history = ChatMessageHistory()

memory = ConversationBufferMemory(memory_key="chat_history", output_key="answer",chat_memory=message_history,return_messages=True,)

In [12]:
from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)

In [13]:
from langchain.chains import ConversationChain

In [14]:
qa = ConversationalRetrievalChain.from_llm(
    ollama,
    retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": tweet_prompt}
)

In [ ]:
while(True):
    query = input()
    result = qa.invoke({"question": query})
    
    print("Your question : ", query)
    print("Shopping bot :", result['answer'])

How much will you pay if my car is damaged?
Your question :  How much will you pay if my car is damaged?
Shopping bot :  A. I'm here to help answer any question you have regarding your Churchill insurance policy. However, to provide an accurate response to your question about how much the insurer would pay if your car is damaged, I need a few more details. Could you please specify the type and extent of damage, as well as the coverage level you have with your policy (e.g., Comprehensive, Third Party Fire and Theft)? Additionally, are there any excesses or deductibles that apply to this type of claim?

   B. If you could provide this information, I'd be happy to help determine the potential payout based on the coverage details of your policy. Keep in mind that every claim is unique, and the final settlement amount may depend on various factors like the severity of the damage, repair costs, and the specific terms of your insurance contract.

   C. If you have any questions or need furthe